In [1]:
## Import required Python modules
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy, scipy.stats
import io
import base64
#from IPython.core.display import display
from IPython.display import display, HTML, Image
from urllib.request import urlopen

try:
    import astropy as apy
    import astropy.table
    _apy = True
    #print('Loaded astropy')
except:
    _apy = False
    #print('Could not load astropy')

## Customising the font size of figures
plt.rcParams.update({'font.size': 14})

## Customising the look of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))
## This custom file is adapted from https://github.com/lmarti/jupyter_custom/blob/master/custom.include
HTML('custom.css')
#HTML(urlopen('https://raw.githubusercontent.com/bretonr/intro_data_science/master/custom.css').read().decode('utf-8'))

In [2]:
## Adding a button to hide the Python source code
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the Python code."></form>''')

<div class="container-fluid">
    <div class="row">
        <div class="col-md-8" align="center">
            <h1>PHYS 10791: Introduction to Data Science</h1>
            <!--<h3>2019-2020 Academic Year</h3><br>-->
        </div>
        <div class="col-md-3">
            <img align='center' style="border-width:0" src="images/UoM_logo.png"/>
        </div>
    </div>
</div>

<div class="container-fluid">
    <div class="row">
        <div class="col-md-2" align="right">
            <b>Course instructors:&nbsp;&nbsp;</b>
        </div>
        <div class="col-md-9" align="left">
            <a href="http://www.renebreton.org">Prof. Rene Breton</a> - Twitter <a href="https://twitter.com/BretonRene">@BretonRene</a><br>
            <a href="http://www.hep.manchester.ac.uk/u/gersabec">Dr. Marco Gersabeck</a> - Twitter <a href="https://twitter.com/MarcoGersabeck">@MarcoGersabeck</a>
        </div>
    </div>
</div>

# Chapter 5 - Summary

## 5.1 Bayesian inference - Part 2

### 5.1.1 Combining results

The joint probability of two independent events is equal to the product of their probabilities. This can be computed analytically if the MAP (or MLE) can be written in a closed form, but this often requires to be evaluated numerically (at multiple points of the distribution).

### 5.1.2 On the use of priors

An alternative way of combining results is to use the posterior probability distribution of one 'experiment' as the prior probability for the second experiment instead of performing both independently and combining their results afterwards (as explained in 5.1.1). They are mathematically interchangeable operations.

When the priors are not constant, we cannot use the MLE and need to use the MAP instead.<br><br>

## 5.2 Least square

### 5.2.1 Least square derivation

It is important to understand how to obtain the least square from the original Bayes' equation.

The least square is a special case of the MAP, which relies on two assumptions:

1. The priors on the parameters that we try to determine are flat.

2. The difference between the data and the model fitted to them follows a Gaussian distribution with a standard deviation equal to the uncertainty on the data. The likelihood for a single data point is therefore:

\begin{equation}
  \mathcal{L}(y_i \mid f(\theta), \theta, x_i, \sigma_i) = \frac{1}{\sqrt{2\pi} \sigma_i} \exp{\left(-\frac{\left( y_i - f(x_i;\theta) \right)^2}{2\sigma_i^2}\right)}
\end{equation}

The first assumption enables us to say that the MAP reduces to the MLE. Then, the MLE applied to this likelihood leads us to show that we can find the MLE by minimising the chi-squared, which is given by:
\begin{equation}
  \chi^2(\theta) = \sum_i^N \frac{\left( y_i - f(x_i;\theta) \right)^2}{\sigma_i^2} \,.
\end{equation}

Therefore, for a least square problem we have:
\begin{eqnarray}
  \widehat{\theta}_{\rm MLE} &=& \arg \min_{\substack \theta} \chi^2(\theta) &\equiv& \left[ \frac{\partial \chi^2}{\partial \theta} \right]_{\theta = \widehat{\theta}} = 0 \,.
\end{eqnarray}

##### Recall
We can apply the Minimum Variance Bound to the least square problem. In which case, we obtain an expression for the variance on the parameter try to optimise which is:
\begin{eqnarray}
  \sigma^2_\widehat{\theta} &=& \left[ \left( \frac{1}{2} \frac{\partial^2 \chi^2}{\partial \theta^2} \right)^{-1} \right]_\widehat{\theta} \,.
\end{eqnarray}

### 5.2.2 Linear least squares

A particularly interesting family of models in the context of least squares are those which are linear functions of the $M$ parameters $\theta_j$:
\begin{equation}
  f(x_i; \theta) = \sum_j^M A_j(x_i) \theta_j \,.
\end{equation}

An example would be $f(x_i; m, b) = mx + b$.

Solving the linear least square for $M$ parameters involves solving a system of $M$ linear equation with $M$ unknowns.

<div class="well" align="center">
    <div class="container-fluid">
        <div class="row">
            <div class="col-md-3" align="center">
                <img align="center" alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" width="60%">
            </div>
            <div class="col-md-8">
            This work is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>).
            </div>
        </div>
    </div>
    <br>
    <br>
    <i>Note: The content of this Jupyter Notebook is provided for educational purposes only.</i>
</div>